In [6]:
import requests
import pandas as pd
import copy
import numpy as np

In [7]:
#!pip install PyPDF2

# import module we'll need to import our custom module
#from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
#copyfile(src = "../input/camarafunctions/pdf_to_text.py", dst = "../working/pdf_to_text.py")

# import all our functions
#from pdf_to_text import Pdf_to_text

# Processando os dados da Câmara

**Autores envolvidos**

In [8]:
def get_autores_informacoes(url):
    nomes = []
    nomes_string = ""
    
    resp = requests.get(url).json()
    
    if("dados" in resp):
        for dado in resp["dados"]:
            if("nome" in dado and isinstance(dado["nome"], str)):
                nomes.append(dado["nome"])
        
        nomes_string = "; ".join(nomes)

    return nomes_string

**Ementa**

In [9]:
def get_ementa(data):
    try:
        return data["ementa"]
    except:
        return ""

**Ano**

In [10]:
def get_ano(data):
    try:
        return data["ano"]
    except:
        return ""

**Data de apresentação**

In [11]:
def get_dataApresentacao(data):
    try:
        return data["dataApresentacao"]
    except:
        return ""

**keywords**

In [12]:
def get_keywords(data):
    try:
        return data["keywords"].lower()
    except:
        return ""

**Autores**

In [13]:
def get_autores(data):
    if("uriAutores" in data):
        return get_autores_informacoes(data["uriAutores"])
    else:
        return ""

**Tema**

In [14]:
def get_tema(prop_id):
    url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/{prop_id}/temas".format(prop_id=prop_id)
    resp = requests.get(url).json()
    
    tema = "" 
    temas = []
        
    if("dados" in resp):
        dados = resp["dados"]
        for dado in dados:
            try:
                if("tema" in dado):
                    temas.append(dado["tema"])
            except:
                continue
        
        if(tema != []):
            tema = "; ".join(temas)
        else:
            tema = ""
            
    
    return tema

**Texto da proposição**

In [15]:
def get_texto(data):
    texto = ""
    
    try:
        if("urlInteiroTeor" in data):
            url = data["urlInteiroTeor"]

            pdf_to_text = Pdf_to_text(url)
            texto = pdf_to_text.extract()
    except:
        texto = ""
        
    return texto

**Proposição**

In [16]:
def get_proposicao_infos(prop_id):   
    ementa = None
    keywords = None
    autores = None
    tema = None
    ano = None
    dataApresentacao = None
    
    print(prop_id, "get_proposicao_infos")
    url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/{prop_id}".format(prop_id=prop_id)
    print("Url", url)

    try:
        resp = requests.get(url).json()
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Wrong: ",err)
        return "ERRO","ERRO","ERRO","ERRO", "ERRO", "ERRO"
    
    try:
        ementa = get_ementa(resp["dados"])
        print(prop_id, "ementa")
        keywords = get_keywords(resp["dados"])
        print(prop_id, "keywords")
        autores = get_autores(resp["dados"])
        print(prop_id, "autores")
        tema = get_tema(prop_id)
        print(prop_id, "tema")
        ano = get_ano(resp["dados"])
        print(prop_id, "ano")
        dataApresentacao = get_dataApresentacao(resp["dados"])
        print(prop_id, "dataApresentacao")
        
    except:
        print("ERRO", prop_id, ":", resp)

    return ementa, keywords, autores, tema, ano, dataApresentacao

**Todas de uma determinada consulta**

In [17]:
def get_proposicoes_ano(url):
    resp = []
    resp1 = requests.get(url).json()
    have_next_page = True

    while(have_next_page):
        have_next_page = False
        
        if("dados" in resp1 and "links" in resp1 and resp1["dados"] != []):
            resp = resp + resp1["dados"]

            for link in resp1["links"]:
                if(link["rel"] == "next"):
                    next_page = link["href"]
                    have_next_page = True
                    break
            
            print("next_page", next_page)
            resp1 = requests.get(next_page).json()
                
    return resp

**Id das proposições de uma faixa**

In [18]:
def get_dados(start, end):
    ano = start
    resp = []

    while(ano <= end):
        url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio={dataInicio}&dataFim={dataFim}&itens={itens}&ordem=ASC&ordenarPor=id".format(dataInicio=str(ano)+"-01-01",dataFim=str(ano)+"-12-31",itens=10000)
        resp_aux = get_proposicoes_ano(url)

        resp = resp + resp_aux

        ano += 1

    return resp

**Dados de todas as proposições na faixa**

In [19]:
def get_resp(ano):
    resp_proposicoes = get_dados(ano, ano)
    print(f"Nro. de proposições no ano de {ano}: {len(resp_proposicoes)}")
    
    return resp_proposicoes
    

## Tratando os dados

In [20]:
def dados_proposicoes(idx, resp, total_threads):
    ementa_list = []
    keywords_list = []
    autores_list = []
    temas_list = []
    id_list = []
    ano_list = []
    dataApresentacao_list = []
    lista_erro = []
    lista_erro2 = []
        
    for dado in resp:
        try:
            ementa, keywords, autores, tema, ano, dataApresentacao = get_proposicao_infos(dado["id"])
            if(ementa == "ERRO"):
                lista_erro2.append(dado["id"])
                continue
            if(ementa is None):
                continue

            ementa_list.append(ementa)
            keywords_list.append(keywords)
            autores_list.append(autores)
            temas_list.append(tema)
            id_list.append(dado["id"])
            ano_list.append(ano)
            dataApresentacao_list.append(dataApresentacao)

        except:
            print("Erro em obter:", dado["id"])
            lista_erro.append(dado["id"])
        
    result_dados_proposicoes = {
        "ementa_list": ementa_list,
        "keywords_list": keywords_list,
        "autores_list": autores_list,
        "temas_list": temas_list,
        "id_list": id_list,
        "ano_list": ano_list,
        "dataApresentacao_list": dataApresentacao_list,
        "lista_erro": lista_erro,
        "lista_erro2": lista_erro2
    }
    
    return result_dados_proposicoes

**Pegando as proposições com erro**

In [21]:
def dados_proposicoes_erro(parameters, idx):
    nova_lista_erro = []
    ementa_list, keywords_list, autores_list, temas_list, id_list, ano_list, dataApresentacao_list, lista_erro, lista_erro2 = parameters["ementa_list"], parameters["keywords_list"], parameters["autores_list"], parameters["temas_list"], parameters["id_list"], parameters["ano_list"], parameters["dataApresentacao_list"], parameters["lista_erro"], parameters["lista_erro2"]
    
    result_dados_proposicoes = copy.deepcopy(parameters)
    
    while(lista_erro != []):
        print("---> ", len(lista_erro))
        for id_proposicao in lista_erro:
            try:
                ementa, keywords, autores, tema, ano, dataApresentacao = get_proposicao_infos(id_proposicao)
                if(ementa == "ERRO"):
                    lista_erro2.append(id_proposicao)
                    continue
                if(ementa is None):
                    continue

                ementa_list.append(ementa)
                keywords_list.append(keywords)
                autores_list.append(autores)
                temas_list.append(tema)
                id_list.append(id_proposicao)
                ano_list.append(ano)
                dataApresentacao_list.append(dataApresentacao)

            except:
                print("Erro em obter:", id_proposicao)
                nova_lista_erro.append(id_proposicao)
                print(dado)

            print("==============")

            lista_erro = copy.deepcopy(nova_lista_erro)
            nova_lista_erro = []
       
    result_dados_proposicoes = {
        "ementa_list": ementa_list,
        "keywords_list": keywords_list,
        "autores_list": autores_list,
        "temas_list": temas_list,
        "id_list": id_list,
        "ano_list": ano_list,
        "dataApresentacao_list": dataApresentacao_list,
        "lista_erro": lista_erro,
        "lista_erro2": lista_erro2
    }
    
    return result_dados_proposicoes

**Threads**

In [22]:
from time import sleep, perf_counter
from threading import Thread

def minera_dados(resp_proposicoes, total_threads):
    def task(idx, resp):
        print(f'Starting the task {idx}...')
        global result_dados_proposicoes

        #Dados das proposições
        result_dados_proposicoes[idx] = dados_proposicoes(idx, resp, total_threads)
        #Proposições que o servidor derrubou
        result_dados_proposicoes[idx] = dados_proposicoes_erro(result_dados_proposicoes[idx], idx)

        #Proposições com outros erros
        result_dados_proposicoes[idx]["lista_erro"] = result_dados_proposicoes[idx]["lista_erro2"]
        result_dados_proposicoes[idx] = dados_proposicoes_erro(result_dados_proposicoes[idx], idx)

        print(f'The task {idx} completed')
    
    #Inicio da coleta
    start_time = perf_counter()

    #Global variable

    global result_dados_proposicoes 
    result_dados_proposicoes = [0]*total_threads

    resp_list = np.array_split(resp_proposicoes, total_threads)

    # create and start threads
    threads = []
    for idx, resp in enumerate(resp_list):
        t = Thread(target=task, args=(idx, resp,))
        threads.append(t)
        t.start()

    # wait for the threads to complete
    for t in threads:
        t.join()

    end_time = perf_counter()

    print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

    return result_dados_proposicoes

**Criando df das proposições**

In [23]:
def gera_df(result_dado, total_threads):
    
    dados = {}

    dados["id"] = []
    dados["ementa"] = []
    dados["keywords"] = []
    dados["autores"] = []
    dados["temas"] = []
    dados["ano"] = []
    dados["dataApresentacao"] = []

    for i in range(0, total_threads):
        dados["id"] += result_dado[i]["id_list"]
        dados["ementa"] += result_dado[i]["ementa_list"]
        dados["keywords"] += result_dado[i]["keywords_list"]
        dados["autores"] += result_dado[i]["autores_list"]
        dados["temas"] += result_dado[i]["temas_list"]
        dados["ano"] += result_dado[i]["ano_list"]
        dados["dataApresentacao"] += result_dado[i]["dataApresentacao_list"]

    df = pd.DataFrame(dados)
    
    return df

**Realiza a mineração**

In [24]:
def run(ano, total_threads):
    resp_proposicoes = get_resp(ano)
    result_dado = minera_dados(resp_proposicoes, total_threads)
    
    df = gera_df(result_dado, total_threads)
    df.to_csv(f"./data/camara_data_{ano}.csv",encoding='utf8', index=False)    

In [25]:
total_threads = 8

anos = [2019, 2018]

for ano in anos:
    print(f"\n\nIteração do ano: {ano}")
    run(ano, total_threads)



Iteração do ano: 2019
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=2&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=3&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=4&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=5&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=6&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&dataFim=2019-12-31&ordem=ASC&ordenarPor=id&pagina=7&itens=100
next_page https://dadosabertos.camara.leg.br/api/v2/proposicoes?dataInicio=2019-01-01&da

KeyboardInterrupt: 